In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#### Args
DATA_IN = "/home/cascade/projects/UrbanHeat/data/" 
FN_STATS = 'processed/AllDATA-GHS-ERA5-HI406-META.csv'
FN_POP = 'interim/GHS-UCDB-Interp.csv'
FN_OUT = DATA_IN+'processed/AllDATA-GHS-ERA5-HI406-FIGDATA.csv'

In [ ]:
STATS = pd.read_csv(DATA_IN+FN_STATS)

In [ ]:
STATS.head()

In [ ]:
STATS[(STATS['ID_HDC_G0'] == 5645) & (STATS['year'] == 1989)]

In [ ]:
# Let's get a subset

df = STATS[STATS['CTR_MN_NM'] == 'France']
print(len(df))

In [ ]:
def group_stats(df):
    "function groups event stats to calc total Tmax days per year for each city"
    
    total_days = df.groupby(['year', 'ID_HDC_G0'])['duration'].sum() # group by city and year
    total_days_df = pd.DataFrame(total_days).reset_index() # reset the index
    total_days_df.rename({'duration': 'total_days'}, axis=1, inplace=True) # rename column 

    return total_days_df

In [ ]:
total_days_df = group_stats(df)
total_days_df.head()
print(len(total_days_df))

In [ ]:
# Match it and check
match = df[['ID_HDC_G0', 'total_days', 'year']]
match.head()

In [ ]:
total_days_df = total_days_df.merge(match, how = 'inner', on = ['ID_HDC_G0', 'year'])

In [ ]:
len(total_days_df)

In [ ]:
# Why is there a difference?
total_days_df['dif'] = total_days_df['total_days_x'] - total_days_df['total_days_y']

In [ ]:
error = total_days_df[total_days_df['dif'] !=0]

In [ ]:
len(error)

In [ ]:
total_days_df

In [ ]:
def df_drop(df, cols):
    "function takes data frame and drops duplicates based on a column, takes list of cols"
    df_out = pd.DataFrame()
    df_out = df.drop_duplicates(cols)
    
    return df_out

In [ ]:
len(total_days_df)
out = df_drop(total_days_df, ('ID_HDC_G0', 'year'))

In [ ]:
len(out)

In [ ]:
len(df.drop_duplicates(('ID_HDC_G0', 'year')))

# Find city IDS w/ 1 data point in 1983 and zeros everywhere else

In [3]:
# Check
FN_FIGDATA = 'processed/AllDATA-GHS-ERA5-HI406-FIGDATA.csv'
FIGDATA = pd.read_csv(DATA_IN+FN_FIGDATA)

In [4]:
FIGDATA.head()

,Unnamed: 0,ID_HDC_G0,year,total_days,P,P1983,P2016,people_days,people_days_heat,people_days_pop
0,0,5645,1983,1,80796.793810,80796.79381,96123.248169,80796.793810,80796.79381,0.000000
1,1,5645,1989,2,78918.494262,80796.79381,96123.248169,157836.988523,161593.58762,-3756.599097
2,2,5645,1990,3,78605.444337,80796.79381,96123.248169,235816.333011,242390.38143,-6574.048420
3,3,5645,1994,1,76481.266889,80796.79381,96123.248169,76481.266889,80796.79381,-4315.526921
4,4,5645,2005,1,80904.416534,80796.79381,96123.248169,80904.416534,80796.79381,107.622724


In [26]:
#### Make a Funtion to reduce
def reduce(df, year, total_days):
    reduce_df = df[(df['year'] == year) & (df['total_days'] == total_days)]
    
    return reduce_df

In [58]:
# Copy all data
df = FIGDATA.copy()

# get years for loop
years = list(range(1984, 2017)) 

for i, year in enumerate(years):
    df_reduce = reduce(df, year, 0) # find 0 total days by year
    ids = list(df_reduce['ID_HDC_G0']) # Get the city ids for total days == 0 for that year
    df = df[df['ID_HDC_G0'].isin(ids)] # subset data by those city ids

In [60]:
np.unique(df['ID_HDC_G0'])

array([ 832, 1732, 5521])